# **Assignment: Performance Optimization and Parallel Computing Exercise**

## **Problem Statement:**

You are given a Python program that performs a large-scale simulation of a physical system. 

The program involves intensive matrix operations, including matrix multiplication, element-wise operations, and solving linear equations. 

The current implementation is slow and not optimized for running on multi-core systems.

##     **Part 1: Initial Profiling and Analysis**

**1. Profiling the Code:**

Profile the given code using cProfile or any other profiling tool of your choice to identify the most time-consuming parts of the program.

Record the functions or parts of the code that are taking the most time.

**2. Memory Usage Analysis:**

Analyze the memory usage of the program. Identify any inefficiencies in memory usage, such as redundant data storage, large temporary arrays, or inefficient memory access patterns.


In [225]:
# Installing the Libraries

!pip install snakeviz
!pip install memory_profiler
!pip install line_profiler
!pip install numba
!pip install numpy
!pip install scipy
!pip install joblib


%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [226]:
# Performance and profiling tools
import cProfile
import pstats
import io
from memory_profiler import memory_usage

# Scientific computing
import numpy as np
import scipy.linalg as la
from numba import njit, prange

# Parallelization and multiprocessing
from joblib import Parallel, delayed, parallel_backend
from multiprocessing import cpu_count

# System and timing
import os
import time

In [227]:
# Set env vars to prevent BLAS libraries from using multiple threads
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

In [228]:
def simulate_physical_system(n):
    A = np.random.rand(n, n)
    B = np.random.rand(n, n)
    C = np.random.rand(n, n)

    # Perform matrix multiplication
    D = np.dot(A, B)

    # Element-wise operations
    E = D * C

    # Solve a linear system
    x = np.linalg.solve(E, np.ones(n))

    return x


In [229]:
profiler = cProfile.Profile()
profiler.enable()
result = simulate_physical_system(1000)
profiler.disable()

s = io.StringIO()
ps = pstats.Stats(profiler, stream=s).sort_stats('cumtime')
ps.print_stats(10)
print(s.getvalue())


         79 function calls in 0.045 seconds

   Ordered by: cumulative time
   List reduced from 42 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.045    0.022 /Users/sm_aswin21/tensorflow-test/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3472(run_code)
        2    0.000    0.000    0.045    0.022 {built-in method builtins.exec}
        1    0.001    0.001    0.045    0.045 /var/folders/3x/7hfm4_jj7zl66x1wm_x_2f5h0000gn/T/ipykernel_53080/1290230547.py:3(<module>)
        1    0.001    0.001    0.044    0.044 /var/folders/3x/7hfm4_jj7zl66x1wm_x_2f5h0000gn/T/ipykernel_53080/648558669.py:1(simulate_physical_system)
        3    0.021    0.007    0.033    0.011 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.021    0.021 <__array_function__ internals>:177(dot)
        1    0.000    0.000    0.012    0.012 <__array_function__ 

####    **Memory Usage Analysis**

In [230]:
def run_and_profile_memory():
    mem_usage, result = memory_usage((simulate_physical_system, (1000,)), retval=True, interval=0.1, timeout=None)
    print(f"Peak memory usage: {max(mem_usage) - min(mem_usage):.2f} MiB")

run_and_profile_memory()


Peak memory usage: 38.52 MiB


####    **Run Line Profiler**

In [231]:
%lprun -f simulate_physical_system simulate_physical_system(1000)


Timer unit: 1e-09 s

Total time: 0.077155 s
File: /var/folders/3x/7hfm4_jj7zl66x1wm_x_2f5h0000gn/T/ipykernel_53080/648558669.py
Function: simulate_physical_system at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def simulate_physical_system(n):
     2         1    3622000.0    4e+06      4.7      A = np.random.rand(n, n)
     3         1    3345000.0    3e+06      4.3      B = np.random.rand(n, n)
     4         1    3460000.0    3e+06      4.5      C = np.random.rand(n, n)
     5                                           
     6                                               # Perform matrix multiplication
     7         1   37749000.0    4e+07     48.9      D = np.dot(A, B)
     8                                           
     9                                               # Element-wise operations
    10         1    1490000.0    1e+06      1.9      E = D * C
    11                                           


####    - If you want to visualise the Profiling Results with snakeviz : 

In [232]:
# !snakeviz profile_stats

In [233]:
profiler.dump_stats('profiling_results.prof')

## **Key Points - Part 1**

####    1. Using cProfile

1. simulate_physical_system takes approximately 0.052 seconds.

2. The np.linalg.solve method is the most time-consuming.

3. np.dot (matrix multiplication) is also significant. 

####    2. Memory Usage Analysis

1. The peak memory usage during the execution of the simulate_physical_system(n) function is reported as: 23.27 MiB

##     **Part 2: Optimization**

####    Code Optimization:

Optimize the identified bottlenecks. You may consider:

- Using high-performance libraries like NumPy, SciPy, or BLAS for matrix operations.
- Refactoring code to reduce unnecessary computations or memory usage.
- Improving memory access patterns to reduce cache misses.

####    Memory Management:

Optimize the memory usage by eliminating unnecessary memory allocations, using views instead of copies where possible, and reducing the memory footprint of large data structures.

In [234]:
import numpy as np
import scipy.linalg as la

def simulate_physical_system_optimized(n):
    """
    Using Lower Precision Data Types:
        
    - Converted all arrays to float32 using .astype(np.float32).
    - Reduces memory usage by half compared to float64.
    - Less memory usage can lead to faster computations due to better cache utilization.
    
    """
    A = np.random.rand(n, n).astype(np.float32)
    B = np.random.rand(n, n).astype(np.float32)
    C = np.random.rand(n, n).astype(np.float32)
    
    
    D = np.dot(A, B)
    del A, B 
    
    # Element-wise operations 
    D *= C 
    del C 
    
    b = np.ones(n)
    try:
        x = la.solve(D, b, assume_a='gen')  # General matrix
    except la.LinAlgError:
        
        print("Matrix is singular; adding regularization.")
        D += np.eye(n) * 1e-8 
        x = la.solve(D, b, assume_a='gen')
    del D  
    
    return x

In [235]:
n = 5000

# Timing the original function

start_time = time.time()
x_original = simulate_physical_system(n)
original_time = time.time() - start_time
print(f"Original function time: {original_time:.4f} seconds")

# Timing the optimized function

start_time = time.time()
x_optimized = simulate_physical_system_optimized(n)
optimized_time = time.time() - start_time
print(f"Optimized function time: {optimized_time:.4f} seconds")

# Verifying results are similar

difference = np.linalg.norm(x_original - x_optimized)
print(f"Difference between results: {difference}")


Original function time: 2.3661 seconds
Optimized function time: 1.7561 seconds
Difference between results: 0.005822214255435301


In [236]:
# original function
def run_and_profile_memory_original():
    mem_usage, _ = memory_usage((simulate_physical_system, (n,)), retval=True, interval=0.1)
    print(f"Original function peak memory usage: {max(mem_usage) - min(mem_usage):.2f} MiB")

run_and_profile_memory_original()

#  optimized function
def run_and_profile_memory_optimized():
    mem_usage, _ = memory_usage((simulate_physical_system_optimized, (n,)), retval=True, interval=0.1)
    print(f"Optimized function peak memory usage: {max(mem_usage) - min(mem_usage):.2f} MiB")

run_and_profile_memory_optimized()

Original function peak memory usage: 1014.44 MiB
Optimized function peak memory usage: 441.84 MiB


In [237]:
# Line profiling for original function
%lprun -f simulate_physical_system simulate_physical_system(n)

# Line profiling for optimized function
%lprun -f simulate_physical_system_optimized simulate_physical_system_optimized(n)


Timer unit: 1e-09 s

Total time: 1.57171 s
File: /var/folders/3x/7hfm4_jj7zl66x1wm_x_2f5h0000gn/T/ipykernel_53080/776204648.py
Function: simulate_physical_system_optimized at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def simulate_physical_system_optimized(n):
     5                                               """
     6                                               Using Lower Precision Data Types:
     7                                                   
     8                                               - Converted all arrays to float32 using .astype(np.float32).
     9                                               - Reduces memory usage by half compared to float64.
    10                                               - Less memory usage can lead to faster computations due to better cache utilization.
    11                                               
    12                                           

##     **Part 3: Parallelization**

**1. Parallelizing the Code:**

- Parallelize the most computationally expensive parts of the code using either Python’s multiprocessing module or OpenMP in C/C++ (if applicable). If you choose to use Python, consider using mpi4py or Joblib for parallelism.
- Ensure that the parallel implementation is efficient and scales well with the number of cores or processes.

**2. Handling Race Conditions and Synchronization:**

- If your parallel code involves shared resources or data, ensure that you handle synchronization issues such as race conditions. 
- Use appropriate locking mechanisms or atomic operations to prevent these issues.

In [238]:
!pip install --upgrade joblib


In [239]:
import joblib
print(joblib.__version__)


1.2.0


In [240]:
import numpy as np
from joblib import Parallel, delayed, parallel_backend

def compute_chunk(A_chunk, B):
    # Compute the chunk of matrix multiplication
    return np.dot(A_chunk, B)

def simulate_physical_system_parallel(n, n_jobs=-1):
    """
    Parallelized version using joblib for multiprocessing.
    """
    # Control BLAS threading to prevent oversubscription
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ['NUMEXPR_NUM_THREADS'] = '1'

    # Generate random matrices with lower precision
    A = np.random.rand(n, n).astype(np.float64)
    B = np.random.rand(n, n).astype(np.float64)
    C = np.random.rand(n, n).astype(np.float64)

    if n_jobs == -1:
        n_jobs = cpu_count()

    # Split A into chunks along the first axis
    A_chunks = np.array_split(A, n_jobs, axis=0)
    del A  # Free memory

    with parallel_backend('loky'):
        results = Parallel(n_jobs=n_jobs, backend='threading')(
            delayed(compute_chunk)(A_chunk, B) for A_chunk in A_chunks
    )


    del B  # Free memory

    # Combine results
    D = np.vstack(results)
    del results

    # Element-wise multiplication (in-place)
    D *= C
    del C

    b = np.ones(n, dtype=np.float64)
    try:
        x = la.solve(D, b)
    except la.LinAlgError:
        D += np.eye(n, dtype=np.float64) * 1e-8
        x = la.solve(D, b)
    del D

    return x


In [241]:
n_jobs = cpu_count()  # Number of CPU cores

# Timing the parallelized function
start_time = time.time()
x_parallel = simulate_physical_system_parallel(n, n_jobs=n_jobs)
parallel_time = time.time() - start_time
print(f"Parallelized function time: {parallel_time:.4f} seconds")


Parallelized function time: 2.9520 seconds


###     **Part 4: Testing and Reporting**

**1. Testing:**

- Test the optimized and parallelized code on different input sizes and record the performance improvements. 

- Ensure that the output remains correct and consistent with the original implementation.

In [242]:
def test_functions(n_values, n_jobs=-1):
    times = {'n': [], 'original': [], 'optimized': [], 'parallel': []}
    correctness = {'n': [], 'difference_opt': [], 'difference_par': []}

    for n in n_values:
        print(f"\nTesting with n = {n}")

        # Original function
        start_time = time.time()
        x_original = simulate_physical_system(n)
        original_time = time.time() - start_time
        print(f"Original function time: {original_time:.4f} seconds")

        # Optimized function
        start_time = time.time()
        x_optimized = simulate_physical_system_optimized(n)
        optimized_time = time.time() - start_time
        print(f"Optimized function time: {optimized_time:.4f} seconds")

        # Parallelized function
        start_time = time.time()
        x_parallel = simulate_physical_system_parallel(n, n_jobs=n_jobs)
        parallel_time = time.time() - start_time
        print(f"Parallelized function time: {parallel_time:.4f} seconds")

        # Record times
        times['n'].append(n)
        times['original'].append(original_time)
        times['optimized'].append(optimized_time)
        times['parallel'].append(parallel_time)

        # Verify correctness
        difference_opt = np.linalg.norm(x_original - x_optimized)
        difference_par = np.linalg.norm(x_original - x_parallel)
        correctness['n'].append(n)
        correctness['difference_opt'].append(difference_opt)
        correctness['difference_par'].append(difference_par)
        print(f"Difference between original and optimized outputs: {difference_opt}")
        print(f"Difference between original and parallel outputs: {difference_par}")

    return times, correctness


In [243]:
# Set input sizes (be cautious with large n; adjust as needed)
n_values = [500, 1000, 1500] 
n_jobs = cpu_count()

# Run tests
times, correctness = test_functions(n_values, n_jobs=n_jobs)



Testing with n = 500
Original function time: 0.1210 seconds
Optimized function time: 0.0956 seconds
Parallelized function time: 0.2645 seconds
Difference between original and optimized outputs: 0.0784786195561539
Difference between original and parallel outputs: 0.08418336757106508

Testing with n = 1000
Original function time: 0.0608 seconds
Optimized function time: 0.2930 seconds
Parallelized function time: 0.3258 seconds
Difference between original and optimized outputs: 0.1874898379576723
Difference between original and parallel outputs: 0.01894753238945993

Testing with n = 1500
Original function time: 0.1488 seconds
Optimized function time: 0.3598 seconds
Parallelized function time: 0.4085 seconds
Difference between original and optimized outputs: 0.025028789971290613
Difference between original and parallel outputs: 0.018167471587858603


###     **Generating Profiling Outputs:**

*   To obtain the profiling outputs for the original, optimized, and parallelized code, here you can find it:



###  **1. Saving Profiling Output**

####    **A. For the Original Code:**

In [244]:
profiler = cProfile.Profile()
profiler.enable()
# Run your original function
result = simulate_physical_system(n)
profiler.disable()

# Save the profiling stats to a file
profiler.dump_stats('original_profile.prof')

# Save readable stats to a text file
with open('original_profile.txt', 'w') as f:
    ps = pstats.Stats(profiler, stream=f)
    ps.sort_stats('cumtime').print_stats()

####    **B. For the Optimized Code:**

In [245]:
profiler = cProfile.Profile()
profiler.enable()
# Run your optimized function
result = simulate_physical_system_optimized(n)
profiler.disable()

# Save the profiling stats to a file
profiler.dump_stats('optimized_profile.prof')

# Save readable stats to a text file
with open('optimized_profile.txt', 'w') as f:
    ps = pstats.Stats(profiler, stream=f)
    ps.sort_stats('cumtime').print_stats()


####    **C. For the Parallel Code:**

In [246]:
profiler = cProfile.Profile()
profiler.enable()
# Run your optimized function
result = simulate_physical_system_parallel(n)
profiler.disable()

# Save the profiling stats to a file
profiler.dump_stats('parallel_profile.prof')

# Save readable stats to a text file
with open('parallel_profile.txt', 'w') as f:
    ps = pstats.Stats(profiler, stream=f)
    ps.sort_stats('cumtime').print_stats()


####    **2. Saving Memory Profiling Output:**

In [247]:
from memory_profiler import memory_usage
import time

n_values = [1000, 2000, 3000]  

functions_to_profile = [
    ('simulate_physical_system', simulate_physical_system),
    ('simulate_physical_system_optimized', simulate_physical_system_optimized),
    ('simulate_physical_system_parallel', simulate_physical_system_parallel)
]

def run_and_profile_memory(func_name, func, n):
    
    print(f"Profiling memory for {func_name} with n = {n}")
    mem_usage, result = memory_usage((func, (n,)), retval=True, interval=0.1)
    filename = f'memory_profile_{func_name}_n{n}.txt'
    with open(filename, 'w') as f:
        for usage in mem_usage:
            f.write(f"{usage}\n")
    peak_memory = max(mem_usage) - min(mem_usage)
    print(f"Peak memory usage for {func_name} (n={n}): {peak_memory:.2f} MiB\n")


for n in n_values:
    for func_name, func in functions_to_profile:
        run_and_profile_memory(func_name, func, n)


Profiling memory for simulate_physical_system with n = 1000
Peak memory usage for simulate_physical_system (n=1000): 24.92 MiB

Profiling memory for simulate_physical_system_optimized with n = 1000
Peak memory usage for simulate_physical_system_optimized (n=1000): 13.34 MiB

Profiling memory for simulate_physical_system_parallel with n = 1000
Peak memory usage for simulate_physical_system_parallel (n=1000): 0.95 MiB

Profiling memory for simulate_physical_system with n = 2000
Peak memory usage for simulate_physical_system (n=2000): 83.19 MiB

Profiling memory for simulate_physical_system_optimized with n = 2000
Peak memory usage for simulate_physical_system_optimized (n=2000): 0.14 MiB

Profiling memory for simulate_physical_system_parallel with n = 2000
Peak memory usage for simulate_physical_system_parallel (n=2000): 91.14 MiB

Profiling memory for simulate_physical_system with n = 3000
Peak memory usage for simulate_physical_system (n=3000): 134.44 MiB

Profiling memory for simulate

/var/folders/3x/7hfm4_jj7zl66x1wm_x_2f5h0000gn/T/ipykernel_53080/776204648.py:27: LinAlgWarning: Ill-conditioned matrix (rcond=2.28062e-08): result may not be accurate.
  x = la.solve(D, b, assume_a='gen')  # General matrix


Peak memory usage for simulate_physical_system_parallel (n=3000): 48.27 MiB



####    **3. Visualize the Profiling Results:**



*   After running the profiling, you can visualize the results using snakeviz.

*   This will open an interactive visualization in your web browser, helping you analyze which parts of your code consume the most time.

In [248]:
! snakeviz original_profile.prof

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fsm_aswin21%2FDesktop%2FAssignment_3%2Foriginal_profile.prof
^C

Bye!


In [187]:
! snakeviz optimized_profile.prof

snakeviz: error: the path /Users/sm_aswin21/Desktop/Assignment_3/optimized_profile.prof does not exist

usage: snakeviz [-h] [-v] [-H ADDR] [-p PORT] [-b BROWSER_PATH] [-s] filename

Start SnakeViz to view a Python profile.

positional arguments:
  filename              Python profile to view

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit
  -H ADDR, --hostname ADDR
                        hostname to bind to (default: 127.0.0.1)
  -p PORT, --port PORT  port to bind to; if this port is already in use a free
                        port will be selected automatically (default: 8080)
  -b BROWSER_PATH, --browser BROWSER_PATH
                        name of webbrowser to launch as described in the
                        documentation of Python's webbrowser module:
                        https://docs.python.org/3/library/webbrowser.html
  -s, --server          start SnakeViz in server-only mode--n

In [190]:
! snakeviz parallel_profile.prof

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fsm_aswin21%2FDesktop%2FAssignment_3%2Fparallel_profile.prof
^C

Bye!
